In [1]:
import sys
import os

from sklearn.linear_model import SGDClassifier, LinearRegression, Lasso, Ridge
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
import seaborn as sn
import random
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from sklearn.manifold import TSNE
import tqdm
import copy
from sklearn.svm import LinearSVC 

from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import TruncatedSVD
import torch
from sklearn.linear_model import SGDClassifier

from sklearn.svm import LinearSVC

import sklearn
from sklearn.linear_model import LogisticRegression
import random
import pickle
import matplotlib.pyplot as plt
from sklearn import cluster
from sklearn import neural_network
from gensim.models.keyedvectors import Word2VecKeyedVectors
from gensim.models import KeyedVectors
import numpy as np
import warnings
import argparse
from sklearn.neural_network import MLPClassifier
from collections import defaultdict
import scipy
from scipy import stats
from scipy.stats import pearsonr
import pandas as pd
from collections import Counter

In [4]:
class Args():
    
    model_type='bert'
    run_id=1
    pickle_file_train='gender_data_for_inlp_train.pickle'
    pickle_file_dev='gender_data_for_inlp_dev.pickle'
    
args=Args()

In [8]:
def load_data(args, file):
    
    with open(file, "rb") as f:
        data=pickle.load(f)
        z=np.array([1 if d["g"]=="f" else 0 for d in data])
        stereotypes=np.array([d["s"] for d in data])
        txts=[d["text"] for d in data]
    
    return z, txts, stereotypes, data

def load_data_representations(args, file, seed=0):
    
    X=np.load("encodings/{}_{}.npy".format(args.model_type, file))
    
    with open("pca/pca_{}_{}.pickle".format(args.model_type, args.run_id), "rb") as f:
        pca=pickle.load(f)
    X=pca.transform(X)
    
    return X

def load_projections(proj_type, seed=0):
    
    with open("interim/{}/{}/Ps_{}.pickle".format(args.model_type, proj_type, proj_type), "rb") as f:
        rank2P=pickle.load(f)
        
        return rank2P             

In [14]:
z_train,txts_train,stereotypes_train,train = load_data(args, args.pickle_file_train)
z_test,txts_test,stereotypes_test,test = load_data(args, args.pickle_file_dev)
z_dev,txts_dev,stereotypes_dev,dev = load_data(args, args.pickle_file_dev)

if False and os.path.exists("analysis/") and os.path.exists("analysis/mode2x.pickle") and os.path.exists("analysis/mode2p.pickle"):
    with open("analysis/mode2p.pickle", "rb") as f:
        mode2p = pickle.load(f)
    with open("analysis/mode2x.pickle", "rb") as f:
        mode2x = pickle.load(f)
        
else:
    if not os.path.exists("analysis"): os.mkdir("analysis")
    mode2x = defaultdict(dict)
    mode2p = defaultdict(dict)
    
    mode2x["train"]=load_data_representations(args, args.pickle_file_train)
    mode2x["dev"]=load_data_representations(args, args.pickle_file_dev)
    mode2x["test"]=load_data_representations(args, args.pickle_file_dev)
    
    for projtype in ["rlace", "inlp"]:
        rank2P=load_projections(projtype)
        mode2p[projtype]=rank2P
        
    with open("analysis/mode2x.pickle", "wb") as f:
        pickle.dump(mode2x, f)
    with open("analysis/mode2p.pickle", "wb") as f:
        pickle.dump(mode2p, f)